In [2]:
!nvidia-smi

Thu Jul 29 09:37:42 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
#config consts
DATASET =     {'train':'train.tar.gz',
               'test':'test.tar.gz',
               'train_mask':'train_mask.tar.gz'}

METADATA=     {'image_level':'train_image_level.csv',
               'study_level':'train_study_level.csv',
               'df_meta':'df_meta.csv',
               'df_fold_rand830':'df_fold_rand830.csv',
               'train_dup':'duplicate.txt',
               'sample_sub':'sample_submission.csv',
               'df_annotate':'df_annotate.csv',
               'df_meta_hw':'df_meta_hw.csv',
               'yolov5m':'yolov5m.pt',
               'yolov5x':'yolov5x.pt'}

CFGMODEL_DIR_DICT = {'B3_512':'effb3-full-512-mask-v8/',
                     'B3_512_PNG': 'effb3-512-png-mask/',
                     'B4_512':'effb4-full-512-mask/',
                     'B5_600':'effb5-600-mask/',
                     'B5_640':'effb5-640-mask/',
                     'B5_640_M2': 'effb5-640-mask/',
                     'D121_640':'d121-640-mask/',
                     'D201_640':'d201-640-mask/',
                     'YOLO5_640':'custom-yolo5-640-v4/',
                     'YOLO5X_768':'yolov5x-768/'}

INPUT_DIR ='/content/drive/My\ Drive/kaggle/covid19-det/input/'

OUTPUT_DIR = {'BSL':'/content/drive/My\ Drive/kaggle/covid19-det/output/',
              'NORM':'/content/drive/My Drive/kaggle/covid19-det/output/'}

IMPORT_DIR = '/content/drive/My Drive/kaggle/covid19-det/nbs/py/'

HENGCK_IM_DIR=IMPORT_DIR+'hengck_code/dummy_01q/'
HENGCK_IM_DIR_YOLO=IMPORT_DIR+'hengck_code/dummy_01a/'


WORK_DIR='/content/'

DATASET_DIR_DICT = {'256': INPUT_DIR+'256_jpg/',
                    '512': INPUT_DIR+'512_jpg/',
                    '512_PNG': INPUT_DIR+'512_png/',
                    '600': INPUT_DIR+'600_jpg/',
                    '640': INPUT_DIR+'640_jpg/',
                    '640_M2':INPUT_DIR+'640_jpg_m2/',
                    '768':INPUT_DIR+'768_jpg/'}

EXPERIMENT='SZ640_YOLO_TEST_WBF_200'
EXPERIMENT_DIR = OUTPUT_DIR['BSL'] + EXPERIMENT+'/'
CFGMODEL_DIR = CFGMODEL_DIR_DICT['YOLO5_640']
MASK_SIZE=(40,40)
L1_FACTOR=4
LR_FACTOR=0.1
MAX_LR_PLATEAU=10
DATASET_DIR = DATASET_DIR_DICT['640']
METADATA_DIR = INPUT_DIR+'metadata/'
FOLDS_SET=[0]
INITIAL_CHECKPOINTS=[None for i in range(5)]

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install pydicom
!pip install ensemble-boxes
#!pip install madgrad
#!pip install timm

import sys
sys.path.append(HENGCK_IM_DIR_YOLO)
sys.path.append(HENGCK_IM_DIR_YOLO+CFGMODEL_DIR)

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from common import *

from siim import *

from lib.net.lookahead import *
from lib.net.radam import *
#from madgrad import MADGRAD

from model import *
from dataset import *
from ensemble_boxes import *

def copy_dataset(ds_dict, ds_dir, work_dir):
  for record in ds_dict:
    print('copy', ds_dir+ds_dict[record], ' to', work_dir)
    !cp {ds_dir+ds_dict[record]} {work_dir}
    print('mkdir',work_dir+record)
    !mkdir {work_dir+record}
    print ('tar -xzf',work_dir+ds_dict[record],'-C',work_dir+record)
    !tar -xzf  {work_dir+ds_dict[record]} -C {work_dir+record}
    print ('rm ',work_dir+ds_dict[record])
    !rm {work_dir+ds_dict[record]}
def copy_metadata(md_dict,md_dir,work_dir):
  for record in md_dict:
    print('copy ', md_dir+md_dict[record],' to ',work_dir)
    !cp {md_dir+md_dict[record]} {work_dir}
copy_dataset(DATASET,DATASET_DIR, WORK_DIR)
copy_metadata(METADATA,METADATA_DIR,WORK_DIR)
!mkdir {EXPERIMENT_DIR}
!ls /content/

     |████████████████████████████████| 1.9 MB 5.0 MB/s 
matplotlib.get_backend :  module://ipykernel.pylab.backend_inline


/usr/local/lib/python3.7/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /content/drive/My Drive/kaggle/covid19-det/nbs/py/hengck_code/dummy_01a/siim/map_boxes/compute_overlap.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


copy /content/drive/My\ Drive/kaggle/covid19-det/input/640_jpg/train.tar.gz  to /content/
mkdir /content/train
tar -xzf /content/train.tar.gz -C /content/train
rm  /content/train.tar.gz
copy /content/drive/My\ Drive/kaggle/covid19-det/input/640_jpg/test.tar.gz  to /content/
mkdir /content/test
tar -xzf /content/test.tar.gz -C /content/test
rm  /content/test.tar.gz
copy /content/drive/My\ Drive/kaggle/covid19-det/input/640_jpg/train_mask.tar.gz  to /content/
mkdir /content/train_mask
tar -xzf /content/train_mask.tar.gz -C /content/train_mask
rm  /content/train_mask.tar.gz
copy  /content/drive/My\ Drive/kaggle/covid19-det/input/metadata/train_image_level.csv  to  /content/
copy  /content/drive/My\ Drive/kaggle/covid19-det/input/metadata/train_study_level.csv  to  /content/
copy  /content/drive/My\ Drive/kaggle/covid19-det/input/metadata/df_meta.csv  to  /content/
copy  /content/drive/My\ Drive/kaggle/covid19-det/input/metadata/df_fold_rand830.csv  to  /content/
copy  /content/drive/My\ D

In [17]:
#----------------
import torch.cuda.amp as amp

class AmpNet(Net):
    @torch.cuda.amp.autocast()
    def forward(self,*args):
        return super(AmpNet, self).forward(*args)

is_mixed_precision = True  #True #False
#run_check_net()


def train_augment(r):
    image = r['image']
    mask = r['mask']
    annotate = r['annotate']

    if 1:
        for fn in np.random.choice([
            lambda image, mask, annotate : do_random_scale(image, mask, annotate, mag=0.35),
            lambda image, mask, annotate : do_random_stretch_y(image, mask, annotate, mag=0.35),
            lambda image, mask, annotate : do_random_stretch_x(image, mask, annotate, mag=0.35),
            lambda image, mask, annotate : do_random_shift(image, mask, annotate, mag=int(0.35*image_size)),
            lambda image, mask, annotate : (image, mask, annotate)
        ],1):
            image, mask, annotate = fn(image, mask, annotate)

        for fn in np.random.choice([
            lambda image, mask, annotate : do_random_rotate(image, mask, annotate, mag=15),
            lambda image, mask, annotate : do_random_hflip(image, mask, annotate),
            lambda image, mask, annotate : (image, mask, annotate)
        ],1):
            image, mask, annotate = fn(image, mask, annotate)

        # ------------------------
        for fn in np.random.choice([
            lambda image : do_random_intensity_shift_contast(image, mag=[0.5,0.5]),
            lambda image : do_random_noise(image, mag=0.05),
            lambda image : do_random_guassian_blur(image),
            lambda image : do_random_blurout(image, size=0.3, num_cut=3),
            #lambda image : do_random_clahe(image),
            #lambda image : do_histogram_norm(image),
            lambda image : image,
        ],1):
            image = fn(image)

    r['image'] = image
    r['mask'] = mask
    r['annotate'] = annotate
    return r

def do_wbf(predict_flat,nms_objectness_threshold = 0.001,nms_iou_threshold = 0.5,nms_pre_max_num   = 200):
  iou_thr=0.575
  skip_box_thr = nms_objectness_threshold
  batch_size = len(predict_flat)
  detection = []
  for b in range(batch_size):
    p = predict_flat[b]
    i = p[..., 4] > nms_objectness_threshold
    p = p[i]
    num = len(p)
    if num==0:
      det = np.zeros((0,5),np.float32) #None
      detection.append(det)
      continue
    boxes = xywh2xyxy(p[:, :4])
    scores = p[:,4]
    if num > nms_pre_max_num:
      i = scores.argsort(descending=True)[:nms_pre_max_num]
      boxes = boxes[i]
      scores = scores[i]
    labels = torch.zeros_like(scores)
    wbf_boxes, wbf_scores, wbf_labels = weighted_boxes_fusion([(boxes/image_size).tolist()], [scores.tolist()], [labels.tolist()],
                                                                    weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    det = np.concatenate([wbf_boxes,wbf_scores[:,None]],-1)
    detection.append(det)
  return detection

def do_valid_wbf(net, valid_loader,df_valid, df_annotate):
  valid_loss = [0,0]
  valid_num  = 0
  valid_detection = []

  net.eval()
  start_timer = timer()
  for t, batch in enumerate(valid_loader):
      batch_size = len(batch['index'])
      annotate = batch['annotate'].cuda()
      image = batch['image'].cuda()
      onehot = batch['onehot']
      label = onehot.argmax(-1)

      with torch.no_grad():
        predict = data_parallel(net, image)
        predict_inf = infer_prediction(predict)
        predict_flat = pyramid_to_flat(predict_inf)

        detection = do_wbf(
                    predict_flat,
                    nms_objectness_threshold=0.001,
                    nms_iou_threshold=0.5,
                    nms_pre_max_num=200)

        truth = make_truth(annotate)
        loss_cls, loss_box, loss_obj = modified_yolo_loss(predict, truth)
                

      #----------
      valid_num += batch_size
      valid_loss[0] += batch_size*loss_box.item()
      valid_loss[1] += batch_size*loss_obj.item()
      print('\r %8d / %d  %s'%(valid_num, len(valid_loader.dataset),time_to_str(timer() - start_timer,'sec')),end='',flush=True)
      valid_detection.extend(detection)
    
  assert(valid_num == len(valid_loader.dataset))
  #print('')
  #----------------------
  valid_loss[0] = valid_loss[0]/valid_num
  valid_loss[1] = valid_loss[1]/valid_num
    
  detection = valid_detection
  #df_image  = make_df_image(df_valid, detection)

  df_predict = {
                    'ImageID':[],
                    'LabelName':[],
                    'Conf':[],
                    'XMin':[],
                    'XMax':[],
                    'YMin':[],
                    'YMax':[],
                }

  for i,det in enumerate(detection):
                    d = df_valid.iloc[i]
                    for x0, y0, x1, y1, c in det:
                        x0 = int(x0  * d.width)
                        y0 = int(y0  * d.height)
                        x1 = int(x1  * d.width)
                        y1 = int(y1  * d.height)

                        df_predict['ImageID'].append(d.image)
                        df_predict['LabelName'].append(0)
                        df_predict['Conf'].append(c)
                        df_predict['XMin'].append(x0)
                        df_predict['XMax'].append(x1)
                        df_predict['YMin'].append(y0)
                        df_predict['YMax'].append(y1)

  df_predict = pd.DataFrame(df_predict)

  df_truth = {
                    'ImageID':[],
                    'LabelName':[],
                    'XMin':[],
                    'XMax':[],
                    'YMin':[],
                    'YMax':[],
                }
    
  gb = df_annotate.groupby('image_id')

  for i,d in df_valid.iterrows():
      g = gb.get_group(d.image)

      for j,f in g.iterrows():
        if f.class_id==0: continue
        x0 = f.x
        y0 = f.y
        x1 = f.x+f.w
        y1 = f.y+f.h
        df_truth['ImageID'].append(d.image)
        df_truth['LabelName'].append(0)
        df_truth['XMin'].append(x0)
        df_truth['XMax'].append(x1)
        df_truth['YMin'].append(y0)
        df_truth['YMax'].append(y1)

  df_truth = pd.DataFrame(df_truth)

  map, _ = mean_average_precision_for_boxes(df_truth, df_predict, iou_threshold=0.5, exclude_not_in_annotations=False, verbose=False)
    
  return [map, map/6, valid_loss[0], valid_loss[1]]
    
def do_valid(net, valid_loader,df_valid, df_annotate):

    valid_loss = [0,0]
    valid_num  = 0
    valid_detection = []

    net.eval()
    start_timer = timer()
    for t, batch in enumerate(valid_loader):
        batch_size = len(batch['index'])
        annotate = batch['annotate'].cuda()
        image = batch['image'].cuda()
        onehot = batch['onehot']
        label = onehot.argmax(-1)

        with torch.no_grad():
            #with amp.autocast():
                predict = data_parallel(net, image)
                predict_inf = infer_prediction(predict)
                predict_flat = pyramid_to_flat(predict_inf)

                detection = do_non_max_suppression(
                    predict_flat,
                    nms_objectness_threshold=0.001,
                    nms_iou_threshold=0.5,
                    nms_pre_max_num=500,
                    nms_post_max_num=25,)

                truth = make_truth(annotate)
                loss_cls, loss_box, loss_obj = modified_yolo_loss(predict, truth)
                

        #----------
        valid_num += batch_size
        valid_loss[0] += batch_size*loss_box.item()
        valid_loss[1] += batch_size*loss_obj.item()
        print('\r %8d / %d  %s'%(valid_num, len(valid_loader.dataset),time_to_str(timer() - start_timer,'sec')),end='',flush=True)
        valid_detection.extend(detection)
    
    assert(valid_num == len(valid_loader.dataset))
    #print('')
    #----------------------
    valid_loss[0] = valid_loss[0]/valid_num
    valid_loss[1] = valid_loss[1]/valid_num
    
    detection = valid_detection
    #df_image  = make_df_image(df_valid, detection)

    df_predict = {
                    'ImageID':[],
                    'LabelName':[],
                    'Conf':[],
                    'XMin':[],
                    'XMax':[],
                    'YMin':[],
                    'YMax':[],
                }

    for i,det in enumerate(detection):
                    d = df_valid.iloc[i]

                    for x0, y0, x1, y1, c in det:
                        x0 = int(x0 / image_size * d.width)
                        y0 = int(y0 / image_size * d.height)
                        x1 = int(x1 / image_size * d.width)
                        y1 = int(y1 / image_size * d.height)

                        df_predict['ImageID'].append(d.image)
                        df_predict['LabelName'].append(0)
                        df_predict['Conf'].append(c)
                        df_predict['XMin'].append(x0)
                        df_predict['XMax'].append(x1)
                        df_predict['YMin'].append(y0)
                        df_predict['YMax'].append(y1)

    df_predict = pd.DataFrame(df_predict)

    df_truth = {
                    'ImageID':[],
                    'LabelName':[],
                    'XMin':[],
                    'XMax':[],
                    'YMin':[],
                    'YMax':[],
                }
    
    gb = df_annotate.groupby('image_id')

    for i,d in df_valid.iterrows():
                    g = gb.get_group(d.image)

                    for j,f in g.iterrows():
                        if f.class_id==0: continue
                        x0 = f.x
                        y0 = f.y
                        x1 = f.x+f.w
                        y1 = f.y+f.h
                        df_truth['ImageID'].append(d.image)
                        df_truth['LabelName'].append(0)
                        df_truth['XMin'].append(x0)
                        df_truth['XMax'].append(x1)
                        df_truth['YMin'].append(y0)
                        df_truth['YMax'].append(y1)
    df_truth = pd.DataFrame(df_truth)

    map, _ = mean_average_precision_for_boxes(
                    df_truth, df_predict, iou_threshold=0.5, exclude_not_in_annotations=False, verbose=False)
    
    return [map, map/6, valid_loss[0], valid_loss[1]]

In [7]:
INITIAL_CHECKPOINTS

[None, None, None, None, None]

In [8]:
#INITIAL_CHECKPOINTS=[None,None,None,None,'best_model.pth']

In [24]:
def run_train_yolo():
    for fold in FOLDS_SET:
        out_dir = OUTPUT_DIR['NORM']+EXPERIMENT+'/fold%d'%fold

        initial_checkpoint=None
        if INITIAL_CHECKPOINTS[fold] is not None: 
          initial_checkpoint = OUTPUT_DIR['NORM']+EXPERIMENT+'/fold%d/checkpoint/'%fold+INITIAL_CHECKPOINTS[fold]

        best_map  = 0.3  
        start_lr   = 0.001
        min_lr =     0.000005
        batch_size = 16 #14 #22

        num_iteration = 24000
        iter_log    = 500
        iter_valid  = 500
        iter_save   = list(range(0, num_iteration, 500))#1*1000
        a_iter_save = []


        ## setup  ----------------------------------------
        for f in ['checkpoint', 'train', 'valid', 'backup']: os.makedirs(out_dir + '/' + f, exist_ok=True)
        # backup_project_as_zip(PROJECT_PATH, out_dir +'/backup/code.train.%s.zip'%IDENTIFIER)

        log = Logger()
        log.open(out_dir + '/log.train.txt', mode='a')
        log.write('\n--- [START %s] %s\n\n' % (IDENTIFIER, '-' * 64))
        log.write('\t%s\n' % COMMON_STRING)
        log.write('\t__file__ = %s\n' % EXPERIMENT)
        log.write('\tout_dir  = %s\n' % out_dir)
        log.write('\n')

        ## dataset ------------------------------------
        df_annotate, df_train, df_valid = make_fold('train-%d'%fold)
        train_dataset = SiimDataset(df_annotate, df_train, train_augment)#train_augment
        valid_dataset = SiimDataset(df_annotate, df_valid, )

        train_loader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset),
            batch_size = batch_size,
            drop_last   = True,
            num_workers = 0,
            pin_memory  = True,
            worker_init_fn=lambda id: np.random.seed(torch.initial_seed() // 2 ** 32 + id),
            collate_fn  = null_collate,
        )
        valid_loader  = DataLoader(
            valid_dataset,
            sampler = SequentialSampler(valid_dataset),
            batch_size  = batch_size,
            drop_last   = False,
            num_workers = 0,
            pin_memory  = True,
            collate_fn  = null_collate,
        )

        log.write('df_fold_file  : %s\n'%(df_fold_file))
        log.write('train_dataset : \n%s\n'%(train_dataset))
        log.write('valid_dataset : \n%s\n'%(valid_dataset))
        log.write('\n')

        #check_anchors(dataset, model=model, thr=hyp['anchor_t'], imgsz=imgsz)

        ## net ----------------------------------------
        log.write('** net setting **\n')
        if is_mixed_precision:
            scaler = amp.GradScaler()
            net = AmpNet().cuda()
        else:
            net = Net().cuda()


        if initial_checkpoint is not None:
            f = torch.load(initial_checkpoint, map_location=lambda storage, loc: storage)
            start_iteration = f['iteration']
            num_iteration = start_iteration + 10000
            start_epoch = f['epoch']
            state_dict  = f['state_dict']
            start_lr = f['lrate']
            best_map = f['map']
            net.load_state_dict(state_dict,strict=True)  #True
        else:
            start_iteration = 0
            start_epoch = 0


        log.write('net=%s\n'%(type(net)))
        log.write('\tinitial_checkpoint = %s\n' % initial_checkpoint)
        log.write('\n')

        # -----------------------------------------------
        if 0: ##freeze
            for p in net.block0.backbone.parameters(): p.requires_grad = False


        optimizer = Lookahead(RAdam(filter(lambda p: p.requires_grad, net.parameters()),lr=start_lr), alpha=0.5, k=5)
        #optimizer = RAdam(filter(lambda p: p.requires_grad, net.parameters()),lr=start_lr)
        #optimizer = MADGRAD( filter(lambda p: p.requires_grad, net.parameters()), lr=start_lr, momentum= 0.9, weight_decay= 0, eps= 1e-06)
        #optimizer = optim.SGD(filter(lambda p: p.requires_grad, net.parameters()),lr=start_lr, momentum=0.9)


        log.write('optimizer\n  %s\n'%(optimizer))
        log.write('\n')


        ## start training here! ##############################################
        log.write('** start training here! **\n')
        log.write('   fold = %d\n'%(fold))
        log.write('   is_mixed_precision = %s \n'%str(is_mixed_precision))
        log.write('   batch_size = %d\n'%(batch_size))
        log.write('   experiment = %s\n' % EXPERIMENT)
        log.write('                           |-------VALID ----------------|---- TRAIN/BATCH --------------\n')
        log.write('rate    plt    iter  epoch | map    map/6  loss1  loss2  | loss1  loss2  | time          \n')
        log.write('-----------------------------------------------------------------------------------------\n')
                  #0.00000  00    0.00* 0.00  | 0.000  0.000  0.000  0.000  | 0.000  0.000  |  0 hr 00 min

        def message(mode='print'):
            if mode==('print'):
                asterisk = ' '
                loss = batch_loss
            if mode==('log'):
                asterisk = '*' if iteration in a_iter_save else ' '
                loss = train_loss

            text = \
                '%0.5f  %-3d  %5.3f%s %4.2f  | '%(rate,rate_plateau, iteration/10000, asterisk, epoch,) +\
                '%4.3f  %4.3f  %4.3f  %4.3f  | '%(*valid_loss,) +\
                '%4.3f  %4.3f  | '%(*loss,) +\
                '%s' % (time_to_str(timer() - start_timer,'min'))

            return text

        #----
        valid_loss = np.zeros(4,np.float32)
        train_loss = np.zeros(2,np.float32)
        batch_loss = np.zeros_like(train_loss)
        sum_train_loss = np.zeros_like(train_loss)
        sum_train = 0
        loss0 = torch.FloatTensor([0]).cuda().sum()
        loss1 = torch.FloatTensor([0]).cuda().sum()
        loss2 = torch.FloatTensor([0]).cuda().sum()


        start_timer = timer()
        iteration = start_iteration
        epoch = start_epoch
        rate = start_lr
        rate_plateau = 0

        while  iteration < num_iteration:
            if (rate<min_lr): break

            for t, batch in enumerate(train_loader):

                if (iteration % iter_valid == 0):
                        valid_loss = do_valid_wbf(net, valid_loader,df_valid, df_annotate) 
                        
                        if best_map < valid_loss[0]:
                          best_map = valid_loss[0]
                          a_iter_save.append(iteration)
                          rate_plateau=0
                          torch.save({
                            'state_dict': net.state_dict(),
                            'iteration': iteration,
                            'epoch': epoch,
                            'lrate': rate,
                            'map':best_map}, out_dir + '/checkpoint/best_model.pth')
                          
                        else:
                          rate_plateau=rate_plateau+1


                if (iteration % iter_log == 0):
                    print('\r', end='', flush=True)
                    log.write(message(mode='log') + '\n')


                # learning rate schduler ------------
                rate = get_learning_rate(optimizer)

                if (rate_plateau>MAX_LR_PLATEAU):
                  rate_plateau=0
                  rate = rate*LR_FACTOR
                  best_model_pth = out_dir+'/checkpoint/best_model.pth'
                  f = torch.load(best_model_pth, map_location=lambda storage, loc: storage)
                  state_dict  = f['state_dict']
                  iteration = f['iteration']
                  epoch = f['epoch']

                  net.load_state_dict(state_dict,strict=True)
                  del optimizer
                  optimizer = Lookahead(RAdam(filter(lambda p: p.requires_grad, net.parameters()),lr=rate), alpha=0.5, k=5)
                  break

                # one iteration update  -------------
                batch_size = len(batch['index'])
                image    = batch['image'].cuda()
                annotate = batch['annotate'].cuda()
                mask     = batch['mask'].cuda()
                onehot   = batch['onehot'].cuda()
                label    = onehot.argmax(-1)

                #----
                net.train()
                optimizer.zero_grad()

                if is_mixed_precision:
                    with amp.autocast():
                        predict = data_parallel(net, image)

                        truth = make_truth(annotate)
                        loss_cls, loss_box, loss_obj = modified_yolo_loss(predict, truth)

                        loss0 =  loss_cls #not used
                        loss1 =  loss_box
                        loss2 =  loss_obj

                    scaler.scale(loss1+loss2).backward()
                    scaler.unscale_(optimizer)
                    #torch.nn.utils.clip_grad_norm_(net.parameters(), 2)
                    scaler.step(optimizer)
                    scaler.update()


                else :
                    assert(False)
                    print('fp32')
                    predict = data_parallel(net, image)
                    #<todo> loss = ...

                    #(loss0+loss1).backward()
                    (loss1).backward()
                    optimizer.step()




                # print statistics  --------
                epoch += 1 / len(train_loader)
                iteration += 1

                batch_loss = np.array([loss1.item(), loss2.item()])
                sum_train_loss += batch_loss
                sum_train += 1
                if iteration % 100 == 0:
                    train_loss = sum_train_loss / (sum_train + 1e-12)
                    sum_train_loss[...] = 0
                    sum_train = 0

                print('\r', end='', flush=True)
                print(message(mode='print'), end='', flush=True)


                #debug--------------------------
                if 0:
                    def draw_yolo_true_objectness(objectness, truth, b, mode='sum', image_size=None):
                        if image_size is None:
                            image_size=feature_size[0]
                        num_anchor = 3
                        overlay = []
                        for n in range(num_head):
                            s = feature_size[n]
                            o = np.zeros((num_anchor,s,s), np.float32)

                            i = np.where(truth['index'][n][0]==b)[0]
                            num_box = len(i)
                            if num_box>0:
                                a  = truth['index'][n][2][i]
                                gj = truth['index'][n][3][i]
                                gi = truth['index'][n][4][i]
                                o[a, gj, gi] = 1

                            if mode == 'max':  o = o.max(0)
                            if mode == 'min':  o = o.min(0)
                            if mode == 'mean': o = o.mean(0)
                            if mode == 'sum':  o = o.sum(0)

                            o = cv2.resize(o, dsize = (image_size, image_size), interpolation=cv2.INTER_NEAREST)
                            cv2.rectangle(o, (0, 0),  (image_size, image_size), 1, 1)
                            overlay.append(o)

                        overlay = np.hstack(overlay)
                        return overlay

                    def draw_yolo_objectness(objectness, mode='sum',  image_size=None):
                        if image_size is None:
                            num_anchor, image_size, image_size = objectness[0].shape

                        overlay = []
                        for p in objectness:
                            p = p.copy()
                            if mode == 'max':  p = p.max(0)
                            if mode == 'min':  p = p.min(0)
                            if mode == 'mean': p = p.mean(0)
                            if mode == 'sum':  p = p.sum(0)

                            p = cv2.resize(p, dsize=(image_size, image_size), interpolation=cv2.INTER_NEAREST)
                            cv2.rectangle(p, (0, 0), (image_size, image_size), 1, 1)
                            overlay.append(p)

                        overlay = np.hstack(overlay)
                        return overlay

                    #----
                    def draw_yolo_true_box(overlay, truth, b):
                        for n in range(num_head):
                            i = np.where(truth['index'][n][0]==b)[0]
                            num_box = len(i)
                            if num_box==0: continue

                            s = feature_stride[n]
                            true_box = truth['box'][n][i]

                            a  = truth['index'][n][2][i]
                            gj = truth['index'][n][3][i]
                            gi = truth['index'][n][4][i]
                            for k in range(num_box):
                                dx, dy, w, h = true_box[k] #true box at that scale
                                cx=int(s*(gi[k]+dx))
                                cy=int(s*(gj[k]+dy))
                                w = int(s*w)
                                h = int(s*h)
                                x0=cx-w//2
                                y0=cy-h//2
                                x1=cx+w//2
                                y1=cy+h//2
                                cv2.rectangle(overlay,(x0,y0),(x1,y1),(0,0,255),3)

                                #---
                                cx = int((gi[k]+0.5)*s)
                                cy = int((gj[k]+0.5)*s)
                                w = int(anchor_size[n][a[k]][0])
                                h = int(anchor_size[n][a[k]][1])
                                x0 = cx - w // 2
                                y0 = cy - h // 2
                                x1 = cx + w // 2
                                y1 = cy + h // 2
                                cv2.circle(overlay, (cx, cy), 2, (255, 0, 0), -1)
                                cv2.rectangle(overlay, (x0, y0), (x1, y1), (255, 0, 0), 2)


                        return overlay

                    def draw_yolo_box(overlay, box, truth, b):
                        for n in range(num_head):
                            i = np.where(truth['index'][n][0]==b)[0]
                            num_box = len(i)
                            if num_box==0: continue

                            a  = truth['index'][n][2][i]
                            gj = truth['index'][n][3][i]
                            gi = truth['index'][n][4][i]
                            for k in range(num_box):
                                cx, cy, w, h = box[n][a[k],gj[k],gi[k]] #
                                cx = int(cx)
                                cy = int(cy)
                                w = int(w)
                                h = int(h)
                                x0=cx-w//2
                                y0=cy-h//2
                                x1=cx+w//2
                                y1=cy+h//2
                                cv2.rectangle(overlay,(x0,y0),(x1,y1),(0,255,0),2)
                        return overlay


                    #----
                    predict = infer_prediction(predict)
                    predict = [p.data.float().cpu().numpy() for p in predict]
                    objectness =[p[...,4] for p in predict]
                    box = [p[...,:4] for p in predict]

                    truth = {
                        'index': [(
                            m[0].data.cpu().numpy(),
                            m[1].data.cpu().numpy(),
                            m[2].data.cpu().numpy(),
                            m[3].data.cpu().numpy(),
                            m[4].data.cpu().numpy(),
                        ) for m in truth['index']],
                        'label': [m.data.cpu().numpy() for m in truth['label']],
                        'box'  : [m.data.cpu().numpy() for m in truth['box']],
                    }

                    image = image.permute(0,2,3,1).contiguous()
                    image = image.data.cpu().numpy()
                    image = (image*255).astype(np.uint8)

                    mask = mask.permute(0,2,3,1).repeat(1,1,1,3)
                    mask = mask.data.cpu().numpy()
                    mask = (mask*255).astype(np.uint8)

                    for b in range(batch_size):

                        overlay1 = draw_yolo_true_objectness(objectness, truth, b, mode='sum')
                        overlay2 = draw_yolo_objectness([p[b] for p in objectness])

                        overlay3 = mask[b].copy()
                        overlay3 = draw_yolo_true_box(overlay3, truth, b)
                        overlay4 = mask[b].copy()
                        overlay4 = draw_yolo_box(overlay4, [p[b] for p in box], truth, b)



                        # ---
                        image_show('image', image[b], resize=0.5)
                        image_show('mask', mask[b], resize=0.5)
                        #image_show('truth', overlay0, resize=0.5)

                        image_show_norm('true objectness', overlay1, min=0, max=1, resize=2)
                        image_show_norm('objectness', overlay2, min=0, max=1, resize=2)
                        image_show('true box', overlay3, resize=0.5)
                        image_show('box', overlay4, resize=0.5)
                        cv2.waitKey(0)
        log.write('\n')

In [25]:
run_train_yolo()

     1200 / 1247   0 min 50 sec

/usr/local/lib/python3.7/dist-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [0.9189453125, 0.003124237060546875, 0.9189453125, 0.0843505859375].
  warnings.warn("Zero area box skipped: {}.".format(box_part))


0.00100  2    0.250  8.04  | 0.407  0.068  0.050  0.170  | 0.050  0.166  |  0 hr 43 min
0.00100  3    0.300  9.65  | 0.362  0.060  0.050  0.166  | 0.050  0.161  |  0 hr 51 min
0.00100  4    0.350  11.25  | 0.349  0.058  0.049  0.174  | 0.048  0.161  |  0 hr 59 min
0.00100  5    0.400  12.86  | 0.366  0.061  0.049  0.181  | 0.051  0.163  |  1 hr 08 min
0.00100  6    0.450  14.47  | 0.365  0.061  0.050  0.177  | 0.047  0.160  |  1 hr 16 min
0.00100  7    0.500  16.08  | 0.373  0.062  0.048  0.176  | 0.048  0.158  |  1 hr 24 min
0.00100  8    0.550  17.68  | 0.417  0.070  0.048  0.173  | 0.048  0.159  |  1 hr 33 min
       64 / 1247   0 min 02 sec

/usr/local/lib/python3.7/dist-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [0.056243896484375, 0.003124237060546875, 0.056243896484375, 0.0843505859375].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/usr/local/lib/python3.7/dist-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [0.03125, -0.00937652587890625, 0.03125, 0.0718994140625].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/usr/local/lib/python3.7/dist-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [0.056243896484375, -0.00937652587890625, 0.056243896484375, 0.0718994140625].
  warnings.warn("Zero area box skipped: {}.".format(box_part))
/usr/local/lib/python3.7/dist-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [0.08123779296875, -0.00937652587890625, 0.08123779296875, 0.0718994140625].
  warnings.warn("Zero area box skipped: {}.".format(b

      160 / 1247   0 min 06 sec

...

      688 / 1247   0 min 30 sec

/usr/local/lib/python3.7/dist-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [0.9189453125, 0.09063720703125, 0.9189453125, 0.171875].
  warnings.warn("Zero area box skipped: {}.".format(box_part))


0.00001  7    0.700  22.51  | 0.437  0.073  0.046  0.172  | 0.045  0.156  |  5 hr 06 min
0.00001  8    0.750  24.12  | 0.447  0.075  0.046  0.172  | 0.046  0.161  |  5 hr 14 min
      240 / 1247   0 min 11 sec

/usr/local/lib/python3.7/dist-packages/ensemble_boxes/ensemble_boxes_wbf.py:88: UserWarning: Zero area box skipped: [-0.0595703125, 0.024993896484375, 0.2412109375, 0.024993896484375].
  warnings.warn("Zero area box skipped: {}.".format(box_part))


0.00001  9    0.800  25.72  | 0.439  0.073  0.046  0.172  | 0.047  0.158  |  5 hr 23 min
0.00001  10   0.850  27.33  | 0.431  0.072  0.046  0.173  | 0.046  0.160  |  5 hr 32 min
0.00001  11   0.900  28.94  | 0.459  0.077  0.046  0.172  | 0.043  0.161  |  5 hr 40 min



In [21]:
df_annotate, df_train, df_valid = make_fold('train-0')
train_dataset = SiimDataset(df_annotate, df_train, train_augment)#train_augment
valid_dataset = SiimDataset(df_annotate, df_valid, )


valid_loader  = DataLoader(
            valid_dataset,
            sampler = SequentialSampler(valid_dataset),
            batch_size  = 16,
            drop_last   = False,
            num_workers = 0,
            pin_memory  = True,
            collate_fn  = null_collate,
        )
valid_loss = [0,0]
valid_num  = 0
valid_detection = []
if is_mixed_precision:
  scaler = amp.GradScaler()
  net = AmpNet().cuda()
else:
  net = Net().cuda()

net.eval()
start_timer = timer()
for t, batch in enumerate(valid_loader):
      batch_size = len(batch['index'])
      annotate = batch['annotate'].cuda()
      image = batch['image'].cuda()
      onehot = batch['onehot']
      label = onehot.argmax(-1)

      with torch.no_grad():
        predict = data_parallel(net, image)
        predict_inf = infer_prediction(predict)
        predict_flat = pyramid_to_flat(predict_inf)

        detection = do_wbf(
                    predict_flat,
                    nms_objectness_threshold=0.001,
                    nms_iou_threshold=0.5,
                    nms_pre_max_num=10)

        truth = make_truth(annotate)
        loss_cls, loss_box, loss_obj = modified_yolo_loss(predict, truth)
                

      #----------
      valid_num += batch_size
      valid_loss[0] += batch_size*loss_box.item()
      valid_loss[1] += batch_size*loss_obj.item()
      print('\r %8d / %d  %s'%(valid_num, len(valid_loader.dataset),time_to_str(timer() - start_timer,'sec')),end='',flush=True)
      valid_detection.extend(detection)
    
assert(valid_num == len(valid_loader.dataset))
#print('')
#----------------------
valid_loss[0] = valid_loss[0]/valid_num
valid_loss[1] = valid_loss[1]/valid_num
    
detection = valid_detection
  #df_image  = make_df_image(df_valid, detection)
df_predict = {
                    'ImageID':[],
                    'LabelName':[],
                    'Conf':[],
                    'XMin':[],
                    'XMax':[],
                    'YMin':[],
                    'YMax':[],
                }

for i,det in enumerate(detection):
                    d = df_valid.iloc[i]
                    for x0, y0, x1, y1, c in det:
                        x0 = int(x0  * d.width)
                        y0 = int(y0  * d.height)
                        x1 = int(x1  * d.width)
                        y1 = int(y1  * d.height)

                        df_predict['ImageID'].append(d.image)
                        df_predict['LabelName'].append(0)
                        df_predict['Conf'].append(c)
                        df_predict['XMin'].append(x0)
                        df_predict['XMax'].append(x1)
                        df_predict['YMin'].append(y0)
                        df_predict['YMax'].append(y1)

df_predict = pd.DataFrame(df_predict)

df_truth = {
                    'ImageID':[],
                    'LabelName':[],
                    'XMin':[],
                    'XMax':[],
                    'YMin':[],
                    'YMax':[],
                }
    
gb = df_annotate.groupby('image_id')

for i,d in df_valid.iterrows():
      g = gb.get_group(d.image)

      for j,f in g.iterrows():
        if f.class_id==0: continue
        x0 = f.x
        y0 = f.y
        x1 = f.x+f.w
        y1 = f.y+f.h
        df_truth['ImageID'].append(d.image)
        df_truth['LabelName'].append(0)
        df_truth['XMin'].append(x0)
        df_truth['XMax'].append(x1)
        df_truth['YMin'].append(y0)
        df_truth['YMax'].append(y1)

df_truth = pd.DataFrame(df_truth)

map, _ = mean_average_precision_for_boxes(df_truth, df_predict, iou_threshold=0.5, exclude_not_in_annotations=False, verbose=False)
    

       32 / 1247   0 min 00 sec

/usr/local/lib/python3.7/dist-packages/ensemble_boxes/ensemble_boxes_wbf.py:64: UserWarning: X1 < 0 in box. Set it to 0.
  warnings.warn('X1 < 0 in box. Set it to 0.')
/usr/local/lib/python3.7/dist-packages/ensemble_boxes/ensemble_boxes_wbf.py:76: UserWarning: Y1 < 0 in box. Set it to 0.
  warnings.warn('Y1 < 0 in box. Set it to 0.')


      160 / 1247   0 min 03 sec

/usr/local/lib/python3.7/dist-packages/ensemble_boxes/ensemble_boxes_wbf.py:73: UserWarning: X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')


      320 / 1247   0 min 06 sec

/usr/local/lib/python3.7/dist-packages/ensemble_boxes/ensemble_boxes_wbf.py:85: UserWarning: Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')


     1247 / 1247   0 min 24 sec

In [22]:
map

0.0

In [23]:
df_predict

,ImageID,LabelName,Conf,XMin,XMax,YMin,YMax
0,f6293b1c49e2,0,0.898926,2647,2808,508,560
1,f6293b1c49e2,0,0.884766,2684,2865,492,569
2,f6293b1c49e2,0,0.884277,1740,1855,133,167
3,f6293b1c49e2,0,0.882812,2600,2775,543,593
4,f6293b1c49e2,0,0.874512,1703,1901,108,142
...,...,...,...,...,...,...,...
10501,e6215d0188e5,0,0.873535,1116,1248,202,235
10502,e6215d0188e5,0,0.870117,1499,1624,190,243
10503,e6215d0188e5,0,0.869141,980,1235,216,292
10504,e6215d0188e5,0,0.867188,1102,1255,161,194
